In [84]:
import os
import sys
import shutil
from Format import YOLO
import pandas as pd
import numpy as np
from PIL import Image

In [99]:
def get_data(csv_path, img_path):
    try:
        df = pd.read_csv(csv_path)
        data = {}
        label_counter = 0
        for idx, row in df.iterrows():
            filename = row[0]
            if row[0] not in data:
                label_counter = 0
                img = Image.open(os.path.join(img_path, filename))
                img_width = str(img.size[0])
                img_height = str(img.size[1])
                img_depth = 3

                size = {
                    "width": img_width,
                    "height": img_height,
                    "depth": img_depth
                }
                data[filename] = {
                    "size": size,
                    "objects": {}
                }
            bndbox = {
                "xmin": row[2],
                "xmax": row[3],
                "ymin": row[4],
                "ymax": row[5]
            }

            cls = row[1]

            obj_info = {
                "name": cls,
                "bndbox": bndbox
            }

            obj = {
                "num_obj": label_counter + 1,
                str(label_counter): obj_info
            }

            data[filename]["objects"] = {**data[filename]["objects"], **obj}

            label_counter = label_counter + 1
        return True, data
    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]

        msg = "ERROR : {}, moreInfo : {}\t{}\t{}".format(e, exc_type, fname, exc_tb.tb_lineno)

        return False, msg

In [102]:
def prepare_val_dataset(src_csv_path, src_img_path, output_csv_path, output_img_path):
    shutil.rmtree(output_img_path)
    os.makedirs(output_img_path)
    if os.path.isfile(output_csv_path):
        os.remove(output_csv_path)
    df = pd.read_csv(src_csv_path)
    img_names = df['ImageID'].unique()
    random_imgs = np.random.choice(img_names, 100, False)
    flipped_df = df.loc[df['ImageID'].isin(random_imgs)].reset_index(drop=True)
    for filename in random_imgs:
        img = Image.open(os.path.join(src_img_path, filename))
        rotated = img.transpose(Image.Transpose.FLIP_LEFT_RIGHT)
        rotated.save(os.path.join(output_img_path, filename))
        flipped_df.loc[flipped_df['ImageID'] == filename, 'Width'] = img.size[0]
    flipped_df['Width'] = flipped_df['Width'].astype(int)
    flipped_df['XMin'] = flipped_df['Width'] - flipped_df['XMin']
    flipped_df['XMax'] = flipped_df['Width'] - flipped_df['XMax']
    flipped_df['XMin'] = flipped_df['XMin'].astype(str)
    flipped_df['XMax'] = flipped_df['XMax'].astype(str)
    # flipped_df.drop(columns=['Width'], inplace=True)
    flipped_df.to_csv(output_csv_path, index=False)

In [103]:
def main(config):

    if config["datasets"] == "Dataverse":
        yolo = YOLO(os.path.abspath(config["cls_list"]))

        flag, data = get_data(config["label"], config["img_path"])
        if flag == True:
            flag, data = yolo.generate(data)

            if flag == True:
                flag, data = yolo.save(data, config["output_path"], config["img_path"],
                                        config["img_type"], config["manifest_path"])

                if flag == False:
                    print("Saving Result : {}, msg : {}".format(flag, data))

            else:
                print("YOLO Generating Result : {}, msg : {}".format(flag, data))

        else:
            print("BDD Parsing Result : {}, msg : {}".format(flag, data))

    else:
        print("Unkwon Datasets")

if __name__ == '__main__':

    config1 = {
        "datasets": "Dataverse",
        "img_path": "./datasets/images/train",
        "label": "./datasets/labels.csv",
        "img_type": ".jpg",
        "manifest_path": "./",
        "output_path": "./datasets/labels/train/",
        "cls_list": "./data/dataverse.names",
    }
    config2 = {
        "datasets": "Dataverse",
        "img_path": "./datasets/images/val",
        "label": "./datasets/labels_val.csv",
        "img_type": ".jpg",
        "manifest_path": "./",
        "output_path": "./datasets/labels/val/",
        "cls_list": "./data/dataverse.names",
    }

    main(config1)
    prepare_val_dataset('datasets/labels.csv', 'datasets/images/train', 'datasets/labels_val.csv', 'datasets/images/val')
    main(config2)


YOLO Generating:|████████████████████████████████████████| 100.0% (674/674)  Complete


YOLO Saving:   |████████████████████████████████████████| 100.0% (674/674)  Complete


YOLO Generating:|████████████████████████████████████████| 100.0% (100/100)  Complete


YOLO Saving:   |████████████████████████████████████████| 100.0% (100/100)  Complete



# Make sure that image matches the label one-to-one

In [3]:
 import glob, os, shutil

'''
Sometimes your image data set might not match with your label data set.
This code does the folowing
(1) Go through your image data set
(2) Search if the corresponding label file exist in the label data set. 
(3) If not, remove current image
'''


def copy_filter(label_dir,image_dir,target_dir_images,target_dir_labels):
    target_dir_images = os.path.abspath(target_dir_images)
    if os.path.exists(target_dir_images) == False:
        os.makedirs(target_dir_images)
    
    for image in os.listdir(image_dir):
        if image.endswith('jpg'):
            image_name = os.path.splitext(image)[0]

            # Corresponding label file name
            label_name = image_name + '.txt'
            image_path = image_dir + '/' + image_name + '.jpg'
            if os.path.isfile(label_dir + '/' + label_name) == False:
                print(" -- DELETE IMAGE [Label file not found -- ]")
                
                print(image_path)
#                 os.remove(image_path)
#             else:
                target_images=target_dir_images+ '/' + image_name + '.jpg'
                shutil.copy(image_path,target_dir_images )
                print(" --COPY IMAGE "+target_images)


    for label in os.listdir(label_dir):
        if label.endswith('.txt'):
            label_name = os.path.splitext(label)[0]

            # Corresponding label file name
            image_name = label_name + '.jpg'
            label_path = label_dir + '/' + label_name + '.txt'
            if os.path.isfile(image_dir + '/' + image_name) == False:
                print(" -- DELETE LABEL [Image file not found -- ]")
                print(label_path)
#                 os.remove(label_path)
#             else:
                target_labels=target_dir_labels+ '/' + label_name + '.txt'
                shutil.copy(label_path,target_labels )
                print(" --COPY lABELS "+target_labels)


'\nSometimes your image data set might not match with your label data set.\nThis code does the folowing\n(1) Go through your image data set\n(2) Search if the corresponding label file exist in the label data set. \n(3) If not, remove current image\n'

In [5]:
label_dir = '../bdd100k_data/labels'
image_dir = '../bdd100k_data/images/train/'
target_dir_images="../datasets/bdd100k/images/train"
target_dir_labels="../datasets/bdd100k/labels/train"
copy_filter(label_dir,image_dir,target_dir_images,target_dir_labels)

 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//2ca868dc-23ece425.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/2ca868dc-23ece425.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//1c1de1e7-7fbaee01.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/1c1de1e7-7fbaee01.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//3b59c8a5-f0b031cc.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/3b59c8a5-f0b031cc.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//1faaee6b-0430c425.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/1faaee6b-0430c425.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//2f755d79-8a93679e.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/2f755d79-8a93679e.jpg
 -- DELETE IMAG

 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/1bdde543-71bde795.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//2a38eef1-0b662b2a.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/2a38eef1-0b662b2a.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//3dbf3a40-1fe3703a.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/3dbf3a40-1fe3703a.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//2b9652a8-36de92bb.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/2b9652a8-36de92bb.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//3d27f886-6a285dbd.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/3d27f886-6a285dbd.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//2c2c7cef-f19282c2.jpg
 --COPY IMAGE /

 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/0e78c3ba-95b55507.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//2a38eef1-427f8e01.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/2a38eef1-427f8e01.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//0b669851-654711b1.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/0b669851-654711b1.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//3d85d245-4e4ac9a8.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/3d85d245-4e4ac9a8.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//0ab64198-7d54b45b.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/0ab64198-7d54b45b.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//1f8f289b-cb3024af.jpg
 --COPY IMAGE /

 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/3e1a7d8e-216344b5.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//0ce9abb1-04bba55e.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/0ce9abb1-04bba55e.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//2a7d1d96-07b05331.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/2a7d1d96-07b05331.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//0d6b7f11-1bf62abe.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/0d6b7f11-1bf62abe.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//3b9920ba-bb910e26.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/3b9920ba-bb910e26.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//0c7c9049-ad7f8be3.jpg
 --COPY IMAGE /

 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/1d2e677c-cb0b0f6d.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//2a133e69-bc5bbfb7.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/2a133e69-bc5bbfb7.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//1e18dcf8-2d1b984d.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/1e18dcf8-2d1b984d.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//2e54b87c-1fe4f360.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/2e54b87c-1fe4f360.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//3e6c9e36-73f9712f.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/3e6c9e36-73f9712f.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//0e05bcf0-0779d306.jpg
 --COPY IMAGE /

...

 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/00beeb02-ba0790aa.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//2a3e9bf5-c67df63d.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/2a3e9bf5-c67df63d.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//0ec63036-4e4b6c92.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/0ec63036-4e4b6c92.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//3ff9cdb2-fc210566.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/3ff9cdb2-fc210566.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//1d78d5cb-3d9b5d09.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/1d78d5cb-3d9b5d09.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//2b6767ac-0ba85ddc.jpg
 --COPY IMAGE /

 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/3f225bea-adfb6fa3.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//2a27ffe5-f23b69fe.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/2a27ffe5-f23b69fe.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//3f21552a-b3e749e9.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/3f21552a-b3e749e9.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//3cd35738-49e1fe5b.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/3cd35738-49e1fe5b.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//0d8ec51b-a906a896.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/0d8ec51b-a906a896.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//1bddb2cf-46bd3cf3.jpg
 --COPY IMAGE /

 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/1a809d9d-913c8a79.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//2e87bbe7-931b8792.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/2e87bbe7-931b8792.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//2d3a0b08-68b0ab6f.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/2d3a0b08-68b0ab6f.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//0b0b93b3-5bb0e887.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/0b0b93b3-5bb0e887.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//2eec7a7b-fa31c5d6.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/2eec7a7b-fa31c5d6.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//3f137341-a1fb0e9e.jpg
 --COPY IMAGE /

 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/0c8a2c79-3698cdc3.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//1f49cce2-affba551.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/1f49cce2-affba551.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//3d34e1e8-5b3fefd0.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/3d34e1e8-5b3fefd0.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//3c919d92-b0d1990d.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/3c919d92-b0d1990d.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//2f147772-298901d1.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/2f147772-298901d1.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//2c0ea11a-69586abb.jpg
 --COPY IMAGE /

 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/2e54034d-5fb9a7f7.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//1e0beb4d-714c3b2a.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/1e0beb4d-714c3b2a.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//1bc18e09-e066bed7.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/1bc18e09-e066bed7.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//2d9a0dac-224565f4.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/2d9a0dac-224565f4.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//1dea280f-7a3a5728.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/train/1dea280f-7a3a5728.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/train//3da75143-ebfe747e.jpg
 --COPY IMAGE /

In [6]:
label_dir2 = '../bdd100k_data/labels'
image_dir2 = '../bdd100k_data/images/val/'
target_dir_images2="../datasets/bdd100k/images/val"
target_dir_labels2="../datasets/bdd100k/labels/val"
copy_filter(label_dir2,image_dir2,target_dir_images2,target_dir_labels2)

 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b25fd5d3-fa4bfca0.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b25fd5d3-fa4bfca0.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//be8d7ea3-a6236da4.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/be8d7ea3-a6236da4.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b30b460d-9829fd26.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b30b460d-9829fd26.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b251064f-30002542.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b251064f-30002542.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//bd98917f-b19d03f7.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/bd98917f-b19d03f7.jpg
 -- DELETE IMAGE [Label file not fo

 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b857eb06-3c8b7034.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b5110041-5e70bd90.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b5110041-5e70bd90.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//be651ed5-bc7ce2fe.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/be651ed5-bc7ce2fe.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//bb289fdb-b6763201.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/bb289fdb-b6763201.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//bab692c9-35904fdb.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/bab692c9-35904fdb.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b5cd3939-a7dffd2d.jpg
 --COPY IMAGE /Users/as665471/works

 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/bcaf73c1-d7002106.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//bd5bb58b-7087c7f3.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/bd5bb58b-7087c7f3.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//bdec2416-d18fb371.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/bdec2416-d18fb371.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//be986afd-b86569ff.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/be986afd-b86569ff.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b8201476-b9dba8dd.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b8201476-b9dba8dd.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b569f76a-33d451fe.jpg
 --COPY IMAGE /Users/as665471/works

 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b657d595-e1026272.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//bd656502-f04cf4da.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/bd656502-f04cf4da.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b2675b13-d04e4a76.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b2675b13-d04e4a76.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b1d7b3ac-36f2d3b7.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b1d7b3ac-36f2d3b7.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b24071b8-b3ee1196.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b24071b8-b3ee1196.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b60f1db0-cded4772.jpg
 --COPY IMAGE /Users/as665471/works

 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/bbf3d83d-d6265b06.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//ba0b31f2-c45b879b.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/ba0b31f2-c45b879b.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//ba362b8d-991d1837.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/ba362b8d-991d1837.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//bc46dc69-94e689ae.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/bc46dc69-94e689ae.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//bb6264e8-a591635c.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/bb6264e8-a591635c.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//bd4518ef-be6a7922.jpg
 --COPY IMAGE /Users/as665471/works

...

 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b81d060a-b2203c99.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b3ae380d-76bac333.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b3ae380d-76bac333.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b8efe857-90dec33b.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b8efe857-90dec33b.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b7a9b166-a7b6abec.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b7a9b166-a7b6abec.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//bac25f5a-af4fd9fd.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/bac25f5a-af4fd9fd.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b981e7e3-e965dc31.jpg
 --COPY IMAGE /Users/as665471/works

 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b1d0091f-75824d0d.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//bbe57528-3722fc17.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/bbe57528-3722fc17.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b8a9d382-3bd7d3e4.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b8a9d382-3bd7d3e4.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b5a72f23-dfdc4b86.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b5a72f23-dfdc4b86.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b39fe3cd-2a77c651.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b39fe3cd-2a77c651.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//bb0ea699-3d008331.jpg
 --COPY IMAGE /Users/as665471/works

 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b215943a-10e44587.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b1ee702d-0ae1fc10.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b1ee702d-0ae1fc10.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//bc07d865-f526e4d9.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/bc07d865-f526e4d9.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//be8d7ea3-4b798531.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/be8d7ea3-4b798531.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b896b680-ce014e4d.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b896b680-ce014e4d.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b35dac08-031e3f6f.jpg
 --COPY IMAGE /Users/as665471/works

 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b3d0d68d-cc5961d6.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//bcde71de-ac5c8c74.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/bcde71de-ac5c8c74.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b3450498-dbbb787e.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b3450498-dbbb787e.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b4201bd2-75c1998d.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b4201bd2-75c1998d.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b9e4cb3d-98ed4484.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b9e4cb3d-98ed4484.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//bbff5146-286eae73.jpg
 --COPY IMAGE /Users/as665471/works

 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b568371d-db4fb582.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//be9ce27e-65ec1c4e.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/be9ce27e-65ec1c4e.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//ba70dfe3-4fd952a1.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/ba70dfe3-4fd952a1.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//bea3ece7-e73287ff.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/bea3ece7-e73287ff.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//b26ba07a-203e440c.jpg
 --COPY IMAGE /Users/as665471/workspace-py/yolo5/datasets/bdd100k/images/val/b26ba07a-203e440c.jpg
 -- DELETE IMAGE [Label file not found -- ]
../bdd100k_data/images/val//bc05c126-b5aff837.jpg
 --COPY IMAGE /Users/as665471/works

In [7]:
labels_list=glob.glob("../datasets/bdd100k/labels/val"+"/"+"*.txt")
images_list=glob.glob("../datasets/bdd100k/images/val"+"/"+"*.jpg")
print(len(labels_list))
print(len(images_list))

labels_list2=glob.glob("../datasets/bdd100k/labels/train"+"/"+"*.txt")
images_list2=glob.glob("../datasets/bdd100k/images/train"+"/"+"*.jpg")
print(len(labels_list2))
print(len(images_list2))

5000
5000
10000
10000


In [ ]:
# copy test_img folder
!cp -r bdd100k_images/images/test/ bdd100k/images/